<!--NAVIGATION-->


<a href="https://colab.research.google.com/github/saskeli/x/blob/master/clustering.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

|                                     -                                     |                                     -                                     |                                     -                                     |
|---------------------------------------------------------------------------|---------------------------------------------------------------------------|---------------------------------------------------------------------------|
|   [Exercise 5 (plant clustering)](<#Exercise-5-(plant-clustering&#41;>)   | [Exercise 6 (nonconvex clusters)](<#Exercise-6-(nonconvex-clusters&#41;>) |      [Exercise 7 (binding sites)](<#Exercise-7-(binding-sites&#41;>)      |



## ML: Clustering

Clustering is one of the types of unsupervised learning. It is similar to classification: the aim is to give a label to each data point. However, unlike in classification, we are not given any examples of labels associated with the data points. We must infer from the data, which data points belong to the same cluster. This can be achieved using some notion of distance between the data points. Data points in the same cluster are somehow close to each other.

One of the simplest clustering methods is the *k-means clustering*. It aims at producing a clustering that is optimal in the following sense:

* the *centre of each cluster* is the average of all points in the cluster
* any point in a cluster is closer to its centre than to a centre of any other cluster

The k-means clustering is first given the wanted number of clusters, say k, as a *hyperparameter*. Next, to start the algorithm, k points from the data set are chosen randomly as cluster centres. Then the following phases are repeated iteratively:

* any data point is set to belong to a cluster, whose centre is closest to it
* then for each cluster a new centre is chosen as the average of the data points in the cluster

This procedure is repeated until the clusters no longer change. This kind of algorithm is called an Expectation-Maximization (EM) algorithm, which is known to converge.

#### <div class="alert alert-info">Exercise 5 (plant clustering)</div>

Using the same iris data set that you saw earlier in the classification, apply k-means clustering with 3 clusters.
Create a function `plant_clustering` that loads the iris data set, clusters the data and returns the accuracy_score.

<hr/>

In [1]:


from itertools import permutations
import scipy
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans


def find_permutation(n_clusters, real_labels, labels):
    permutation=[]
    for i in range(n_clusters):
        idx = labels == i
        # Choose the most common label among data points in the cluster
        new_label=scipy.stats.mode(real_labels[idx])[0][0]
        permutation.append(new_label)
    return permutation

def plant_clustering():
    X, y = load_iris(return_X_y=True)
    model = KMeans(3, random_state= 0)
    model.fit(X)
    permutation = find_permutation(3, y, model.labels_)
    new_labels = [permutation[label] for label in model.labels_]

    return accuracy_score(y, new_labels)

def main():
    print(plant_clustering())

if __name__ == "__main__":
    main()


0.8933333333333333


#### <div class="alert alert-info">Exercise 6 (nonconvex clusters)</div>

This exercise can give four points at maximum!

Read the tab separated file data.tsv from the `src` folder into a DataFrame. The dataset has two features X1 and X2, and the label y. Cluster the feature matrix using DBSCAN with different values for the eps parameter. Use values in `np.arange(0.05, 0.2, 0.05)` for clustering. For each clustering, collect the accuracy score, the number of clusters, and the number of outliers. Return these values in a DataFrame, where columns and column names are as in the below example.

Note that DBSCAN uses label -1 to denote outliers , that is, those data points that didn't fit well in any cluster. You have to modify the find_permutation function to handle this: ignore the outlier data points from the accuracy score computation. In addition, if the number of clusters is not the same as the number of labels in the original data, set the accuracy score to NaN.

         eps   Score  Clusters  Outliers                             
    0    0.05      ?         ?         ?
    1    0.10      ?         ?         ?
    2    0.15      ?         ?         ?
    3    0.20      ?         ?         ?

Before submitting the solution, you can plot the data set (with clusters colored) to see what kind of data we are dealing with.

Points are given for each correct column in the result DataFrame.
<hr/>

In [ ]:

import scipy
import pandas as pd
import numpy as np
from sklearn import cluster
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

def find_permutation(n_clusters, real_labels, labels):
    permutation=[]
    for i in range(n_clusters):
        idx = labels == i
        # Choose the most common label among data points in the cluster
        new_label=scipy.stats.mode(real_labels[idx])[0][0]
        permutation.append(new_label)
    return permutation

def nonconvex_clusters():
    df = pd.read_csv("src/data.tsv", sep = "\t")
    X= df[["X1", "X2"]]
    y= df["y"]
    label_count = len(y.unique())
    eps = np.arange(0.05,0.2,0.05)
    
    final = []

    for i, val in enumerate(eps):
        model = DBSCAN(eps=val)
        model.fit(X)
        cluster_count = len(set(model.labels_)) - (1 if -1 in model.labels_ else 0)
        outlier_count = np.count_nonzero(model.labels_ == -1)
        non_outliers = model.labels_ !=-1
        permutation = find_permutation(cluster_count, y[non_outliers], model.labels_[non_outliers])
        new_labels = [permutation[label] for label in model.labels_[non_outliers]]
        if label_count != cluster_count:
            score = np.nan
        else:
            score= accuracy_score(y[non_outliers], new_labels)

        
        final.append([val, score, cluster_count, outlier_count])

    plt.scatter(X["X1"], X["X2"],c= model.labels_)
    plt.show()
    return pd.DataFrame(data=final, columns = ["eps", "Score", "Clusters", "Outliers"], dtype="float")

def main():
    print(nonconvex_clusters())
    #x= nonconvex_clusters()
    


if __name__ == "__main__":
    main()


#### <div class="alert alert-info">Exercise 7 (binding sites)</div>

This exercise can give three points at maximum!

A binding site is a piece of DNA where a certain protein prefers to bind. The piece of DNA can be described as a string consisting of letters A, C, G, and T, which correspond to nucleotides Adenine, Cytosine, Guanine, and Thymine.
In this exercise the length of binding sites is eight nucleotides. They are stored in the file `data.seq`,
and the binding sites there are classified into two classes.

Part 1. Write function `toint` that converts a nucleotide to an integer. Use the following mapping:
```
A -> 0
C -> 1
G -> 2
T -> 3
```

Write also function `get_features_and_labels` that gets a filename as a parameter. The function should load the contents of the file into a DataFrame. The column `X` contains a string. Convert this column into a feature matrix using the above `toint` function. For example the column `["GGATAATA","CGATAACC"]` should result to the feature matrix
```
[[2,2,0,3,0,0,3,0],
[1,2,0,3,0,0,1,1]]
```
The function should return a pair, whose first element is the feature matrix and the second element is the label vector.

Part 2. Create function `cluster_euclidean` that gets a filename as parameter. Get the features and labels using the function from part 1. Perform hierarchical clustering using the function `sklearn.cluster.AgglomerativeClustering`. Get two clusters using `average` linkage and `euclidean` affinity. Fit the model and predict the labels. Note that you may have to use the `find_permutation` function again, because even though the clusters are correct, they may be labeled differently than the real labels given in `data.seq`. The function should return the accuracy score.

Part 3. Create function `cluster_hamming` that works like the function in part 2, except now using the [hamming](https://en.wikipedia.org/wiki/Hamming_distance) affinity. Even though it is possible to pass the function `hamming` to `AgglomerativeClustering`, let us now compute the Hamming distance matrix explicitly. We can achieve this using the function `sklearn.metrics.pairwise_distances`. Use the affinity parameter `precomputed` to `AgglomerativeClustering`. And give the distance matrix you got from `pairwise_distances`, instead of the feature matrix, to the `fit_predict` method of the model. If you want, you can visualize the clustering using the provided `plot` function.

**NB!** When submitting your solution, please comment away all `plot` function calls. This might cause tests to fail on the server.

Which affinity (or distance) do you think is theoretically more correct of these two (Euclidean or Hamming)? Why?

In [ ]:


from numpy.core.function_base import linspace
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import accuracy_score
from sklearn.metrics import pairwise_distances

from matplotlib import pyplot as plt

import seaborn as sns
sns.set(color_codes=True)
import scipy.spatial as sp
import scipy.cluster.hierarchy as hc
import scipy

def find_permutation(n_clusters, real_labels, labels):
    permutation=[]
    for i in range(n_clusters):
        idx = labels == i
        # Choose the most common label among data points in the cluster
        new_label=scipy.stats.mode(real_labels[idx])[0][0]
        permutation.append(new_label)
    return permutation

def toint(x):
    empty = []
    for c in x:
        if c =="A":
            empty.append(0)
        if c =="C":
            empty.append(1)
        if c =="G":
            empty.append(2)
        if c =="T":
            empty.append(3)
    #print(empty)
    return empty

def get_features_and_labels(filename):
    df = pd.read_csv(filename, sep ="\t")
    empty= []
    for c in df["X"]:
            empty.append(toint(c))
    #X= [c for c in df["X"]]
    return (np.array(empty), np.array(df["y"]))


def plot(distances, method='average', affinity='euclidean'):
    mylinkage = hc.linkage(sp.distance.squareform(distances), method=method)
    g=sns.clustermap(distances, row_linkage=mylinkage, col_linkage=mylinkage )
    g.fig.suptitle(f"Hierarchical clustering using {method} linkage and {affinity} affinity")
    plt.show()

def cluster_euclidean(filename):
    X, y = get_features_and_labels(filename)
    model = AgglomerativeClustering(affinity="euclidean" ,linkage="average")
    model.fit(X)
    permutation = find_permutation(2, y,model.labels_)
    new_labels= [permutation[label] for label in model.labels_]
    return accuracy_score(y,new_labels)

def cluster_hamming(filename):
    X, y = get_features_and_labels(filename)
    dist = pairwise_distances(X, metric="hamming")
    model = AgglomerativeClustering(affinity="precomputed", linkage="average")
    model.fit_predict(dist)
    permutation = find_permutation(2, y,model.labels_)
    new_labels= [permutation[label] for label in model.labels_]
    return accuracy_score(y,new_labels)


def main():
    #print(toint("A"))
    #print(get_features_and_labels("src/data.seq"))
    print("Accuracy score with Euclidean affinity is", cluster_euclidean("src/data.seq"))
    print("Accuracy score with Hamming affinity is", cluster_hamming("src/data.seq"))

if __name__ == "__main__":
    main()


<!--NAVIGATION-->


<a href="https://colab.research.google.com/github/saskeli/x/blob/master/clustering.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
